In [1]:


from tensorflow.keras.models import load_model
import shap
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np

import pandas as pd


In [ ]:

model = load_model("..\\..\\models\\combined_cnn1_bs256_k7_noweights.keras",
                                compile=False)
df = pd.read_pickle("..\\..\\data\\.MIT_Heartbeat_original_fs180hz.pkl")

class_labels = {"N": 0,
                "S": 1,
                "V": 2,
                "F": 3,
                "Q": 4}
record_length = 315
X_l2 = df.iloc[:, 6:6+record_length]
X_v5 = df.iloc[:, 6+record_length:]
X_l2v5 = df.iloc[:, 6:]
X = X_l2
y = df["class"].replace(class_labels)

# X = digital_filter_preprocess(X)
# X = substract_mean(X, n=3)
# X, y = delete_rows_of_class_n(X, y, p=0.5, label=0)
#X = preprocess_fft(X)

X_train_l2, X_test_l2, y_train_labels, y_test_labels = train_test_split(X_l2, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)
X_train_v5, X_test_v5, y_train_labels, y_test_labels = train_test_split(X_v5, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)
X_train_l2v5, X_test_l2v5, y_train_labels, y_test_labels = train_test_split(X_l2v5, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

# One-Hot-Encoding
y_train = to_categorical(y_train_labels)
y_test = to_categorical(y_test_labels)

X1 = np.array(X_test_l2)
X2 = np.array(X_test_v5)

X_test = np.concatenate((X1, X2), axis=1)


In [30]:
list(sigmoid_weights(np.arange(0.001,30)).values())

[0.31002551887238755,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [65]:
values1 = np.load("shapvalues_000_127.npy")
values2 = np.load("shapvalues_128_256.npy")
values3 = np.load("shapvalues_256_384.npy")
shap_values =np.concatenate((values1, values2, values3))
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots

mean_values = np.mean(shap_values, axis=0)
std_upper = mean_values + np.std(shap_values, axis=0)
std_lower = mean_values - np.std(shap_values, axis=0)
time = np.arange(0, 315)
std_lower.shape

(630, 5)

In [61]:

def plot_sigmoid_weights():
    def sigmoid_weights(weights):
        """ Custom modification of the sigmoid function to have a little bit of different values"""
        weights = np.array(weights)
        weights = weights / np.min(weights)
        weights =  1/(1+np.exp(-(weights/10-0.9)))
        weights_dict = {i: weights[i] for i in range(len(weights))}
        return weights_dict
    class_weights = {0: 0.23988963006158529,
    1: 7.638768115942029,
    2: 3.4197891321978915,
    3: 29.694366197183097,
    4: 2.670847189231987}
    distribution = pd.read_pickle("data\\MIT_classdistribution")

    sigmoid = sigmoid_weights(list(class_weights.values()))
    # list(distribution)
    # sigmoid.values()
    import matplotlib.pyplot as plt
    import plotly.graph_objects as go
    x = np.arange(0, 30)
    fig = go.Figure()
    FONT = dict(family="Arial", size=20,)
    FONT_TITLE = dict(family="Arial", size=22)

    x = np.linspace(np.min(list(class_weights.values())), np.max(list(class_weights.values())))
    fig.add_trace(go.Scatter(x=x, y=list(sigmoid_weights(x).values()),
                            showlegend=False,
                            line=dict(width=2, color='blue'),
                            ))
    fig.add_trace(go.Scatter(x=list(class_weights.values()),
                            y=list(sigmoid.values()),
                            hovertext=[ f"Class {i}" for i in list(class_weights.keys())],
                            marker=dict(
                                    symbol='x',  # Use 'x' as the marker symbol
                                    size=10,     # Size of the marker
                                    color='red', # Color of the marker
                                    line=dict(
                                        width=2,
                                        color='black'  # Border color of the marker
                                    )),
                            mode="markers",
                            showlegend=False

                            ))
    fig.update_layout(
        title=dict(text="Sample Weights", font=FONT_TITLE),
        xaxis_title=dict(text="Balanced Weights", font=FONT),
        yaxis_title=dict(text="Sigmoid Weights", font=FONT),
        template = "ggplot2",
        width=500,
        height=350,
    )
    return fig




In [37]:
np.max(list(class_weights.values()))

29.694366197183097

In [31]:
X_test = np.concatenate((X1, X2), axis=1)
X_test = shap.sample(X_test, 512, random_state=42)
X_test = X_test[0:128, :]
i = 0
shap_values = np.load("shapevalues_0_127.npy")
shap.decision_plot(2, shap_values[i,:,:], X_test[i, :], feature_names=X.columns)


ValueError: The feature_names arg must include all features represented in shap_values.

In [13]:
shap_values.shape

(128, 630, 5)